In [1]:
import numpy as np
import rioxarray as rxr
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
from osgeo import gdal_array

tf.get_logger().setLevel('ERROR')

2022-03-08 07:32:50.908082: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
model = tf.keras.models.load_model('/adapt/nobackup/projects/ilab/projects/Vietnam/Jordan/VIETNAM_PRIORITY/clouds-binary-mixed-std/58-0.04.hdf5')
net_input_shape = model.input_shape[1:]
net_output_shape = model.output_shape[1:]
net_output_shape = (256, 256, 1)
print(net_input_shape, net_output_shape)

2022-03-08 07:33:07.721697: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-08 07:33:07.721903: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-08 07:33:07.772145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:62:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-03-08 07:33:07.772192: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-08 07:33:07.772231: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-03-08 07:33:07.772257: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2

(512, 512, 4) (256, 256, 1)


In [3]:
%%time
image_filename = '/att/gpfsfs/briskfs01/ppl/mwooten3/Vietnam_LCLUC/TOA/M1BS/pansharpen_TamNongClip/WV02_20130125_M1BS_103001001DD53500-toa_pansharpen_clip.tif'
# image_filename = '/adapt/nobackup/projects/ilab/projects/Vietnam/Jordan/VIETNAM_PRIORITY/tree_model_data/data/Keelin02_20130125_data.tif'
image = rxr.open_rasterio(image_filename, cache=True)
image = image.transpose("y", "x", "band")

CPU times: user 69.5 ms, sys: 10.4 ms, total: 79.9 ms
Wall time: 146 ms


In [6]:
image.shape

(38702, 71223, 4)

In [ ]:
%%time
image = tf.convert_to_tensor(image)
image = tf.expand_dims(image, 0)
print(image.shape, image.device) 

In [ ]:
image[0, 512:1024, 512:1024, :]

In [ ]:

#tf.image.extract_patches(
#    images, sizes, strides, rates, padding, name=None
#)

In [ ]:
chunks = tf.image.extract_patches(image, [1, net_input_shape[0], net_input_shape[1], 1],
                                  [1, net_output_shape[0], net_output_shape[1], 1],
                                  [1, 1, 1, 1], padding='VALID')
print(chunks.shape) 

In [ ]:
with tf.device('/cpu:0'):
    chunks = tf.reshape(chunks, (-1,) + net_input_shape)
chunks.shape

In [ ]:
%%time
for i in range(0, chunks.shape[0], 4):
    #print(chunks[i:i+4].shape)
    model.predict_on_batch(chunks[i:i+4])   

In [ ]:
"""
def _predict_array(self, data: np.ndarray, image_nodata_value):
        
        Runs model on data.
        Parameters
        ----------
        data: np.ndarray
            Block of image to apply the model to.
        image_nodata_value: dtype of data
            Nodata value in image. If given, nodata values are
            replaced with nans in output.
        Returns
        -------
        np.ndarray:
            Result of applying model to data.
        
        net_input_shape = self._model.input_shape[1:]
        net_output_shape = self._model.output_shape[1:]

        image = tf.convert_to_tensor(data)
        image = tf.expand_dims(image, 0)

        assert net_input_shape[2] == data.shape[2],\
               'Model expects %d input channels, data has %d channels' % (net_input_shape[2], data.shape[2])

        # supports variable input size, just toss everything in
        if net_input_shape[0] is None and net_input_shape[1] is None:
            result = np.squeeze(self._model.predict_on_batch(image))
            if not result.flags['WRITEABLE']: # older tensorflow version
                result = np.array(result)
            if image_nodata_value is not None:
                x0 = (data.shape[1] - result.shape[1]) // 2
                y0 = (data.shape[0] - result.shape[0]) // 2
                invalid = (data if len(data.shape) == 2 else \
                          data[:, :, 0])[y0:y0 + result.shape[0], x0:x0 + result.shape[1]] == image_nodata_value
                if len(result.shape) == 2:
                    result[invalid] = math.nan
                else:
                    result[invalid, :] = math.nan
            return result

        out_shape = (data.shape[0] - net_input_shape[0] + net_output_shape[0],
                     data.shape[1] - net_input_shape[1] + net_output_shape[1])

        out_type = tf.dtypes.as_dtype(self._model.dtype)
        chunks = tf.image.extract_patches(image, [1, net_input_shape[0], net_input_shape[1], 1],
                                          [1, net_output_shape[0], net_output_shape[1], 1],
                                          [1, 1, 1, 1], padding='VALID')
        chunks = tf.reshape(chunks, (-1,) + net_input_shape)

        best = np.zeros((chunks.shape[0],) + net_output_shape, dtype=out_type.as_numpy_dtype)
        # do 8 MB at a time... this is arbitrary, may want to change in future
        BATCH_SIZE = max(1, int(8 * 1024 * 1024 / net_input_shape[0] / net_input_shape[1] /
                                net_input_shape[2] / out_type.size))
        for i in range(0, chunks.shape[0], BATCH_SIZE):
            best[i:i+BATCH_SIZE] = self._model.predict_on_batch(chunks[i:i+BATCH_SIZE])

        retval = np.zeros(out_shape + (net_output_shape[-1],))
        for chunk_idx in range(0, best.shape[0]):
            r = (chunk_idx // (  out_shape[1] // net_output_shape[1])) * net_output_shape[0]
            c = (chunk_idx  % ( out_shape[1] // net_output_shape[1])) * net_output_shape[1]
            retval[r:r+net_output_shape[0],c:c+net_output_shape[1],:] = best[chunk_idx,:,:,:]

        if image_nodata_value is not None:
            ox = (data.shape[1] - out_shape[1]) // 2
            oy = (data.shape[0] - out_shape[0]) // 2
            output_slice = data[oy:-oy, ox:-ox, 0]
            retval[output_slice == image_nodata_value] = math.nan

        return retval
"""